<a href="https://colab.research.google.com/github/cypherix/ssd.pytorch/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
os.chdir('drive/My Drive/try')

In [0]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np

In [0]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [0]:
def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

In [0]:
cfg = pedestrian
batch_size = 16
gamma = 0.1
dataset_root = VOC_ROOT
save_folder = 'weights/'
basenet = 'vgg16_reducedfc.pth'
lr = 1e-4
momentum = 0.9
weight_decay = 5e-4

In [0]:
def train(start_iter=0):
    
    #Dataset
    dataset = VOCDetection(root = dataset_root,transform=SSDAugmentation(cfg['min_dim'], MEANS))
    
    #Building ssd
    ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
    net = ssd_net
    
    #Activating Cuda
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True
    
    print('Resuming training, loading {}...'.format('weights/ssd300_VOC_1000.pth'))
    ssd_net.load_weights('weights/ssd300_VOC_'+str(start_iter)+'.pth')
    
    #Loading weights
    #vgg_weights = torch.load(save_folder + basenet)
    #print('Loading base network...')
    #ssd_net.vgg.load_state_dict(vgg_weights)
    #print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    #ssd_net.extras.apply(weights_init)
    #ssd_net.loc.apply(weights_init)
    #ssd_net.conf.apply(weights_init)
    
    net = net.cuda()
    
    #Loss and optimization
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum = momentum,weight_decay = weight_decay)
    criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,False, True)
    
    net.train()
    # loss counters
    loc_loss = 0
    conf_loss = 0
    epoch = 0
    print('Loading the dataset...')

    epoch_size = len(dataset) // batch_size
    print('Training SSD on:', dataset.name)
    
    step_index = 0
    
    #Loading Dataset    
    data_loader = data.DataLoader(dataset, batch_size, num_workers=4, shuffle=True,
                                  collate_fn=detection_collate, pin_memory=True)
    
    # create batch iterator
    batch_iterator = iter(data_loader)
    for iteration in range(start_iter, cfg['max_iter']):
        #if iteration in cfg['lr_steps']:
        #    step_index += 1
        #    adjust_learning_rate(optimizer, gamma, step_index)

        # load train data
        try:
            images, targets = next(batch_iterator)
        except StopIteration:
            batch_iterator = iter(data_loader)
            images, targets = next(batch_iterator)
        
        images = Variable(images.cuda())
        with torch.no_grad():
            targets = [Variable(ann.cuda()) for ann in targets]
        
        # forward
        t0 = time.time()
        out = net(images)
        
        # backprop
        optimizer.zero_grad()
        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c
        loss.backward()
        optimizer.step()
        t1 = time.time()
        loc_loss += loss_l.item()
        conf_loss += loss_c.item()

        if iteration % 100 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.item()), end=' ')

        
        if iteration != 0 and iteration % 1000 == 0:
            print('\nSaving state, iter:', iteration)
            torch.save(ssd_net.state_dict(), 'weights/ssd300_VOC_' + repr(iteration) + '.pth')
    
    torch.save(ssd_net.state_dict(),
               save_folder + 'ssd_person.pth')

In [0]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [0]:
def xavier(param):
    init.xavier_uniform_(param)

In [0]:
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

In [0]:
if __name__ == '__main__':
    train(138000)

Resuming training, loading weights/ssd300_VOC_1000.pth...
Loading weights into state dict...
Finished!
Loading the dataset...
Training SSD on: VOC0712


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 19.3302 sec.
iter 138000 || Loss: 1.8315 || 
Saving state, iter: 138000
timer: 0.4816 sec.
iter 138100 || Loss: 1.9744 || timer: 0.3870 sec.
iter 138200 || Loss: 2.5221 || timer: 0.4103 sec.
iter 138300 || Loss: 2.1575 || timer: 0.4117 sec.
iter 138400 || Loss: 2.3497 || timer: 0.4177 sec.
iter 138500 || Loss: 1.6133 || timer: 0.4502 sec.
iter 138600 || Loss: 2.3512 || timer: 0.4471 sec.
iter 138700 || Loss: 2.0656 || timer: 0.4534 sec.
iter 138800 || Loss: 2.3042 || timer: 0.4324 sec.
iter 138900 || Loss: 1.8358 || timer: 0.4391 sec.
iter 139000 || Loss: 2.2822 || 
Saving state, iter: 139000
timer: 0.4550 sec.
iter 139100 || Loss: 2.5102 || timer: 0.4469 sec.
iter 139200 || Loss: 2.6075 || timer: 0.4385 sec.
iter 139300 || Loss: 2.1186 || timer: 0.4475 sec.
iter 139400 || Loss: 2.1126 || timer: 0.4319 sec.
iter 139500 || Loss: 2.9796 || timer: 0.4146 sec.
iter 139600 || Loss: 2.3358 || timer: 0.4203 sec.
iter 139700 || Loss: 1.9458 || timer: 0.4375 sec.
iter 139800 || Loss: 2.1

In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!LS

/bin/bash: LS: command not found


In [0]:
!LS

/bin/bash: LS: command not found


In [0]:
!LS

/bin/bash: LS: command not found


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!LS

/bin/bash: LS: command not found


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!la

/bin/bash: la: command not found


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
 !ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:

!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py


In [0]:
!ls

 data	   output		    ssd_person.ipynb   test_videos   utils
 eval.py  'person with hog.ipynb'   ssd.py	       train.ipynb   weights
 layers    __pycache__		    test.py	       train.py
